# Analisi - Grafi

In [1]:
import pickle
from shapely import Point
import geopandas as gpd

from my_paths import *
import Graph_functions

## Analisi Automatica - Documentazione

Tramite la funzione Graph_functions.auto_analysis_poi() è possibile eseguire in automatico la ricreazione del grafo stradale con il dizionario dei pesi desiderato. Spiegazione della funzione:
- custom_weights: Un dizionario di pesi del tipo:
    ```python
    {"cycleway": 0, "primary": 1, ....}
    ```
    Possibilità di inserire come chiave: "default": quando non viene trovata una "highway" associta al dizionario,  
    verrà utilizzato il valore contenuto in "default".  
    In questo modo se si fornisce un dizionario in cui c'è soltanto la chiave "default", tutte le strade avranno quel  
    valore associato (esempio per creare un grafo di strade senza pesi):
    ```python
    {"default": 1}
    ```
    Se non viene dato nessun dizionario di pesi, verrà utilizzato direttamente il grafo salvato in staging, che contiene i pesi
    standard che abbiamo stabilito, salvati su file json.
- Si possono inserire i vari percorsi di salvataggio dei file pickle (grafo) e geojson(geoDataFrame).
    Richiede una struttura a liste di dizionari di geoDataFrame per "poi" del tipo:
    ```python
    gdf_list = [
        {"gdf":  gpd.read_file(PATH_DEL_GEOJSON),
        "tipo": "tipo che si vuole assegnare agli archi che connettono il gdf al grafo",
        "attr": {"nome_attributo": "valore_attributo"} # Opzionale, di solito non ci serve
        }
    ]
    ```

## Estensione rete ciclabili

In [28]:
gdf_ciclabili = gpd.read_file(PATH_STRADE_CICLABILI_GEOJSON_STAGING)

Provo a creare dei percorsi che hanno lo scopo di unire tutti i frammenti di ciclabili esistenti.
Quindi userò come "poi" tutti i nodi delle "highway"=cycleway

In [29]:
gdf_ciclabili_cycleway = gdf_ciclabili[gdf_ciclabili["highway"] == "cycleway"]
gdf_point_ciclabili = []
for idx, row in gdf_ciclabili_cycleway.iterrows():
    gdf_point_ciclabili.append(Point(row.geometry.coords[0]))
    gdf_point_ciclabili.append(Point(row.geometry.coords[-1]))
gdf_point_ciclabili = gpd.GeoDataFrame(gdf_point_ciclabili, columns=["geometry"], crs=CRS_GRAD)

Faccio partire il calcolo dei percosi

In [30]:
gdf_list = [
    {"gdf":  gdf_point_ciclabili,
     "tipo": "poi_ciclabili"}
]
Graph_functions.auto_analysis_poi(gdf_list, PATH_GEOJSON=PATH_EXTENDED_CICLABILI_CLEAN)
# Rimuoviamo gli archi artificiali che tanto in questo caso specifico sono archi di punti collegati con loro stessi
# Che quindi non rappresentano nulla, sono solo rumore
gdf_extended_ciclabili = gpd.read_file(PATH_EXTENDED_CICLABILI_CLEAN)
gdf_extended_ciclabili = gdf_extended_ciclabili[gdf_extended_ciclabili["artificial"]==False]
gdf_extended_ciclabili.reset_index(drop=True).to_file(PATH_EXTENDED_CICLABILI_CLEAN, driver="GeoJSON")

### Statistiche

In [31]:
gdf_extended_ciclabili = gpd.read_file(PATH_EXTENDED_CICLABILI_CLEAN)

1. Km per ogni "highway"

In [32]:
gdf_extended_ciclabili_highway = gdf_extended_ciclabili.to_crs(CRS_METR).groupby("highway").agg({
    "length": lambda x: sum(x/1000)
    }).sort_values(by="length", ascending=False)
print(gdf_extended_ciclabili_highway.head(6))
print(gdf_extended_ciclabili_highway["length"].sum(), "km")

                 length
highway                
cycleway     209.282437
footway       36.254350
residential   15.552589
secondary     13.979275
tertiary      11.675173
service        5.011441
308.9554463607366 km


2. Km non di "ciclabile", quindi da costruire

In [33]:
float(gdf_extended_ciclabili[gdf_extended_ciclabili["ciclabile"] == "False"]["length"].sum()/1000)

93.91261168478297

## Sport e tempo libero

Utilizzeremo i seguenti dati:
1. **Parchi**
2. **Impianti sportivi**
3. **Aree gioco**  

Saranno i nostri "poi", cioè "Point of Interest" che verranno aggiunti al Grafo della rete Ciclabile/Stradale per la ricerca di percorsi

In [2]:
gdf_list = [
    {"gdf":  gpd.read_file(PATH_PARCHI_CLEAN),
     "tipo": "parchi",
     "attr": {"priorita": 1}},
     {"gdf":  gpd.read_file(PATH_IMPIANTI_SPORTIVI_CLEAN),
     "tipo": "impianti_sportivi",
     "attr": {"priorita": 2}},
     {"gdf":  gpd.read_file(PATH_AREE_GIOCO_CLEAN),
     "tipo": "aree_gioco",
     "attr": {"priorita": 3}} 
]
Graph_functions.auto_analysis_poi(gdf_list,
                                  PATH_PICKLE=PATH_SPORT_TEMPO_LIBERO_ANALISI_PICKLE_STAGING,
                                  PATH_GEOJSON=PATH_SPORT_TEMPO_LIBERO_ANALISI_CLEAN)

Proviamo a richiedere la versione "ridotta" dei poi

In [7]:
with open(PATH_SPORT_TEMPO_LIBERO_ANALISI_PICKLE_STAGING, "rb") as file:
    G_sport = pickle.load(file)

Graph_functions.auto_filter_poi(G_percorsi=G_sport,
                                max_pois=400,
                                peso_inquinamento_vs_incidenti=0,
                                peso_fattori_esterni_vs_distribuzione_poi=1,
                                PATH_PICKLE=PATH_SPORT_TEMPO_LIBERO_ANALISI_PICKLE_STAGING.replace(".pickle", "_filtrato.pickle"),
                                PATH_GEOJSON=PATH_SPORT_TEMPO_LIBERO_ANALISI_CLEAN.replace(".geojson", "_filtrato.geojson"))

### Statisiche

In [13]:
gdf_analisi_sport = gpd.read_file(PATH_SPORT_TEMPO_LIBERO_ANALISI_CLEAN.replace(".geojson", "_filtrato.geojson"))

In [14]:
len(gdf_analisi_sport[gdf_analisi_sport["artificial"] == True])

400

pre pulizia per tutte le statistiche

In [15]:
# Eliminiamo connessioni "artificiali" usate per connettere i "poi" al grafo delle strade
gdf_analisi_sport_mod = gdf_analisi_sport[gdf_analisi_sport["artificial"] != True]

# Eliminiamo i percorsi completamente contenuti all'interno dei "poi" poligonari: parchi
gdf = gpd.read_file(PATH_PARCHI_CLEAN)
geometry_parchi = gdf.union_all()
gdf_senza_parchi = []
for idx, row in gdf_analisi_sport_mod.iterrows():
    if not geometry_parchi.contains(row.geometry):
        gdf_senza_parchi.append(row)
gdf_analisi_sport_mod = gpd.GeoDataFrame(gdf_senza_parchi, crs=CRS_GRAD)

1. Km per "highway"

In [16]:
# Raggruppiamo per "highway" e calcoliamo (in km -> /1000) la lunghezza complessiva delle strade
gdf_analisi_sport_mod_most_highways = gdf_analisi_sport_mod.to_crs(CRS_METR).groupby("highway").agg({
    "length": lambda x: sum(x/1000)
    }).sort_values(by="length", ascending=False)
# Stampiamo il risultato
print(gdf_analisi_sport_mod_most_highways)
print(f"Tot: {gdf_analisi_sport_mod_most_highways["length"].sum()}")

                                 length
highway                                
cycleway                     100.661339
footway                       66.219267
residential                   14.011855
tertiary                      11.996346
secondary                      7.781333
track                          3.195524
pedestrian                     2.116348
primary                        2.018947
service                        1.769118
path                           1.010400
cycleway footway               0.453654
footway service                0.451056
footway pedestrian             0.359980
footway steps                  0.330102
cycleway path                  0.291043
residential secondary          0.277534
residential pedestrian         0.243722
footway path                   0.177522
pedestrian path                0.118616
tertiary_link                  0.117791
residential footway            0.111161
unclassified                   0.107320
residential service            0.094434


2. Km non di "ciclabile", quindi da costruire

In [17]:
float(gdf_analisi_sport_mod[gdf_analisi_sport_mod["ciclabile"] == "False"]["length"].sum()/1000)

112.65026341818994

Numero poi per municipio che sono stati connessi e % poi connessi in ogni municipio rispetto al totale connesso

In [ ]:
gdf_analisi_sport_raggruppato = gdf_analisi_sport.groupby(["MUNICIPIO"]).agg({
    "poi": "sum",
    "length": lambda x: sum(x)/1000
}).reset_index()
gdf_analisi_sport_raggruppato["MUNICIPIO"] = gdf_analisi_sport_raggruppato["MUNICIPIO"].astype(int)
gdf_analisi_sport_raggruppato = gdf_analisi_sport_raggruppato.sort_index()
gdf_analisi_sport_raggruppato = gdf_analisi_sport_raggruppato.merge(
    gpd.read_file(PATH_INCIDENTI_CLEAN)[["MUNICIPIO", "Incidenti"]], on="MUNICIPIO")

gdf_pois = gpd.read_file(PATH_SCORE_X_MUNICIPI)[["MUNICIPIO", "parchi", "impianti_sportivi", "aree_gioco"]]
gdf_pois["tot"] = gdf_pois.drop("MUNICIPIO", axis=1).sum(axis=1)
gdf_pois["MUNICIPIO"] = gdf_pois["MUNICIPIO"].astype(int)
gdf_analisi_sport_raggruppato = gdf_analisi_sport_raggruppato.merge(gdf_pois[["MUNICIPIO", "tot"]], on="MUNICIPIO")
somma_poi = gdf_analisi_sport_raggruppato["poi"].sum()
gdf_analisi_sport_raggruppato["% poi"] = gdf_analisi_sport_raggruppato.apply(lambda row: row["poi"] / somma_poi * 100, axis=1)
gdf_analisi_sport_raggruppato.sort_values(["poi", "Incidenti", "length"], ascending=False, inplace=True)
print(gdf_analisi_sport_raggruppato)


Dettaglio sulla quantità di "tipo di poi"

In [33]:
gdf_analisi_sport = gdf_analisi_sport[gdf_analisi_sport["tipo"] != "Strade_ciclabili"]
gdf_analisi_sport_raggruppato = gdf_analisi_sport.groupby("tipo").agg({
    "poi": "sum"}).reset_index().sort_values(["poi"], ascending=[False])
gdf_analisi_sport_raggruppato

,tipo,poi
0,aree_gioco,263
1,impianti_sportivi,101
2,parchi,36


## Istruzione

Utilizzeremo i seguenti dati:
1. **Biblioteche**
2. **Scuole**
Saranno i nostri "poi", cioè "Point of Interest" che verranno aggiunti al Grafo della rete Ciclabile/Stradale per la ricerca di percorsi

In [ ]:
gdf_list = [
    {"gdf":  gpd.read_file(PATH_BIBLIOTECHE_CLEAN),
     "tipo": "Biblioteche"},
     {"gdf":  gpd.read_file(PATH_SCUOLE_CLEAN),
     "tipo": "Scuole"}
     ]
Graph_functions.auto_analysis_poi(gdf_list, PATH_GEOJSON=PATH_ISTRUZIONE_CLEAN)

In [23]:
gdf_scuole = gpd.read_file(PATH_SCUOLE_CLEAN)
len(gdf_scuole[gdf_scuole["Grado"] == "Scuola primaria"])

218

### Statisiche

In [14]:
gdf_analisi_istruzione = gpd.read_file(PATH_ISTRUZIONE_CLEAN, driver= "GeoJSON")

c:\Matteo\Coding\Project\Progetto-Bike\.venv\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver GeoJSON does not support open option DRIVER
  return ogr_read(


In [15]:
# Eliminiamo connessioni "artificiali" usate per connettere i "poi" al grafo delle strade
gdf_analisi_istruzione_mod = gdf_analisi_istruzione[gdf_analisi_istruzione["artificial"] != True]

# Leggi i due POI
gdf_scuole = gpd.read_file(PATH_SCUOLE_CLEAN)
gdf_biblioteche = gpd.read_file(PATH_BIBLIOTECHE_CLEAN)

# Unisci in un'unica geometria
geometry_poi = gpd.GeoSeries(
    gpd.pd.concat([gdf_scuole.geometry, gdf_biblioteche.geometry], ignore_index=True)
).union_all()

# Filtra il GDF di partenza
gdf_senza_poi = []
for idx, row in gdf_analisi_istruzione_mod.iterrows():
    if not geometry_poi.contains(row.geometry):
        gdf_senza_poi.append(row)

# Crea nuovo GeoDataFrame
gdf_analisi_istruzione_mod = gpd.GeoDataFrame(gdf_senza_poi, crs=CRS_GRAD)

# Raggruppiamo per "highway" e calcoliamo (in km -> /1000) la lunghezza complessiva delle strade
gdf_analisi_istruzione_mod_most_highways = gdf_analisi_istruzione_mod.to_crs(CRS_METR).groupby("highway").agg({
    "length": lambda x: sum(x/1000)
    }).sort_values(by="length", ascending=False)

In [ ]:
# Stampiamo il risultato
print(gdf_analisi_istruzione_mod_most_highways)
print(f"Tot: {gdf_analisi_istruzione_mod_most_highways["length"].sum()}")

## Cultura e spettacolo

Utilizzeremo i seguenti dati: 

1. **Musei**
2. **Cinema**
3. **Teatri**

Saranno i nostri "poi", cioè "Point of Interest"
che verranno aggiunti al Grafo della rete 
Ciclabile/Stradale per la ricerca di percorsi

In [33]:
gdf_list = [
    {"gdf":  gpd.read_file(PATH_MUSEI_CLEAN),
     "tipo": "Musei"},
     {"gdf":  gpd.read_file(PATH_CINEMA_CLEAN),
     "tipo": "Cinema"}, 
     {"gdf":  gpd.read_file(PATH_TEATRI_CLEAN),
     "tipo": "Teatri"}
     ]
Graph_functions.auto_analysis_poi(gdf_list, PATH_GEOJSON=PATH_CULTURA_SPETTACOLO_CLEAN)

In [34]:
gdf_analisi_cultura_spettacolo = gpd.read_file(PATH_CULTURA_SPETTACOLO_CLEAN, driver= "GeoJSON")

c:\Matteo\Coding\Project\Progetto-Bike\.venv\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver GeoJSON does not support open option DRIVER
  return ogr_read(


In [35]:
# Eliminiamo connessioni "artificiali" usate per connettere i "poi" al grafo delle strade
gdf_analisi_cultura_spettacolo_mod = gdf_analisi_cultura_spettacolo[gdf_analisi_cultura_spettacolo["artificial"] != True]

# Leggi i due POI
gdf_musei = gpd.read_file(PATH_MUSEI_CLEAN)
gdf_teatri = gpd.read_file(PATH_TEATRI_CLEAN)
gdf_cinema = gpd.read_file(PATH_CINEMA_CLEAN)

# Unisci in un'unica geometria
geometry_poi = gpd.GeoSeries(
    gpd.pd.concat([gdf_cinema.geometry, gdf_teatri.geometry, gdf_musei.geometry], ignore_index=True)
).union_all()

# Filtra il GDF di partenza
gdf_senza_poi = []
for idx, row in gdf_analisi_cultura_spettacolo.iterrows():
    if not geometry_poi.contains(row.geometry):
        gdf_senza_poi.append(row)

# Crea nuovo GeoDataFrame
gdf_analisi_cultura_spettacolo_mod = gpd.GeoDataFrame(gdf_senza_poi, crs=CRS_GRAD)

# Raggruppiamo per "highway" e calcoliamo (in km -> /1000) la lunghezza complessiva delle strade
gdf_analisi_cultura_spettacolo_mod_most_highways = gdf_analisi_cultura_spettacolo_mod.to_crs(CRS_METR).groupby("highway").agg({
    "length": lambda x: sum(x/1000)
    }).sort_values(by="length", ascending=False)

In [36]:
# Stampiamo il risultato
print(gdf_analisi_cultura_spettacolo_mod_most_highways)
print(f"Tot: {gdf_analisi_cultura_spettacolo_mod_most_highways["length"].sum()}")

                         length
highway                        
cycleway              43.951569
footway               28.989399
residential           11.314465
secondary              7.660920
tertiary               5.699776
pedestrian             2.074184
primary                1.281142
footway steps          0.496667
service                0.442759
cycleway path          0.423736
path                   0.276162
unclassified           0.204832
secondary_link         0.154071
residential cycleway   0.116689
primary_link           0.082050
footway service        0.078886
cycleway footway       0.058601
footway pedestrian     0.048820
steps path             0.018311
Tot: 103.37304195969466


## Sanità

Utilizzeremo i seguenti dati: 

1. **Ospedali**


Saranno i nostri "poi", cioè "Point of Interest"
che verranno aggiunti al Grafo della rete 
Ciclabile/Stradale per la ricerca di percorsi

In [ ]:
gdf_list = [
    {"gdf":  gpd.read_file(PATH_OSPEDALI_CLEAN),
     "tipo": "Ospedali"}
     ]
Graph_functions.auto_analysis_poi(gdf_list,PATH_GEOJSON=PATH_SANITA_CLEAN)

In [38]:
gdf_analisi_ospedali = gpd.read_file(PATH_SANITA_CLEAN, driver= "GeoJSON")

c:\Matteo\Coding\Project\Progetto-Bike\.venv\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver GeoJSON does not support open option DRIVER
  return ogr_read(


In [39]:
# Eliminiamo connessioni "artificiali" usate per connettere i "poi" al grafo delle strade
gdf_analisi_ospedali_mod = gdf_analisi_ospedali[gdf_analisi_ospedali["artificial"] != True]

# Eliminiamo i percorsi completamente contenuti all'interno dei "poi" poligonari: parchi
gdf = gpd.read_file(PATH_OSPEDALI_CLEAN)
geometry_parchi = gdf.union_all()
gdf_senza_poi = []
for idx, row in gdf_analisi_ospedali_mod.iterrows():
    if not geometry_parchi.contains(row.geometry):
        gdf_senza_poi.append(row)
gdf_analisi_ospedali_mod = gpd.GeoDataFrame(gdf_senza_poi, crs=CRS_GRAD)

# Raggruppiamo per "highway" e calcoliamo (in km -> /1000) la lunghezza complessiva delle strade
gdf_analisi_ospedali_mod_most_highways = gdf_analisi_ospedali_mod.to_crs(CRS_METR).groupby("highway").agg({
    "length": lambda x: sum(x/1000)
    }).sort_values(by="length", ascending=False)

In [40]:
# Stampiamo il risultato
print(gdf_analisi_ospedali_mod_most_highways)
print(f"Tot: {gdf_analisi_ospedali_mod_most_highways["length"].sum()}")

                           length
highway                          
cycleway                39.189874
footway                 15.456106
tertiary                 8.819226
residential              4.817837
secondary                4.593407
primary                  4.326607
service                  2.566652
path                     0.572301
pedestrian               0.542666
track                    0.516093
unclassified             0.281732
footway steps            0.187148
residential secondary    0.146541
residential pedestrian   0.146171
track service            0.126378
residential path         0.088589
cycleway footway         0.079023
tertiary_link            0.050109
primary_link             0.012722
Tot: 82.51918564502445


## Statistiche POI x Municipi

In [5]:
# Carica dati, aggiungendo una colonna "tipo" e tenendo solo questa colonna e geometry
municipi = gpd.read_file(PATH_MUNICIPI_CLEAN)
parchi = gpd.read_file(PATH_PARCHI_CLEAN).assign(tipo="parchi")[["tipo", "geometry"]]
fontane = gpd.read_file(PATH_FONTANE_CLEAN).assign(tipo="fontane")[["tipo", "geometry"]]
impianti = gpd.read_file(PATH_IMPIANTI_SPORTIVI_CLEAN).assign(tipo="impianti_sportivi")[["tipo", "geometry"]]
scuole = gpd.read_file(PATH_SCUOLE_CLEAN).assign(tipo="scuole")[["tipo", "geometry"]]
aree_gioco = gpd.read_file(PATH_AREE_GIOCO_CLEAN).assign(tipo="aree_gioco")[["tipo", "geometry"]]
biblioteche = gpd.read_file(PATH_BIBLIOTECHE_CLEAN).assign(tipo="biblioteche")[["tipo", "geometry"]]
stazioni_bikemi = gpd.read_file(PATH_BIKEMI_CLEAN).assign(tipo="stazioni_bikemi")[["tipo", "geometry"]]
case_acqua = gpd.read_file(PATH_CASE_ACQUA_CLEAN).assign(tipo="case_acqua")[["tipo", "geometry"]]
cinema = gpd.read_file(PATH_CINEMA_CLEAN).assign(tipo="cinema")[["tipo", "geometry"]]
farmacie = gpd.read_file(PATH_FARMACIE_CLEAN).assign(tipo="farmacie")[["tipo", "geometry"]]
musei = gpd.read_file(PATH_MUSEI_CLEAN).assign(tipo="musei")[["tipo", "geometry"]]
ospedali = gpd.read_file(PATH_OSPEDALI_CLEAN).assign(tipo="ospedali")[["tipo", "geometry"]]
teatri = gpd.read_file(PATH_TEATRI_CLEAN).assign(tipo="teatri")[["tipo", "geometry"]]

Uniamo le righe dei vari dataframe in uno unico

In [6]:
pois = gpd.pd.concat([
    parchi, fontane, impianti, scuole, aree_gioco, biblioteche,
    stazioni_bikemi, case_acqua, cinema, farmacie, musei, ospedali, teatri
], ignore_index=True)

Tramite sjoin con predicato "intersect": eseguiamo una left-join con il gdf dei municipi in base
all'intersezione delle geometrie

In [7]:
pois_con_municipio = gpd.sjoin(
    municipi,
    pois,
    how="left",
    predicate="intersects"
    ).drop(["index_right"], axis=1) # eliminiamo l'index_right (conserva l'indice del dataframe del gdf di destra ma non ci serve)

Raggruppiamo per Municipio e Tipo (mettiamo anche geometry così ci rimane per il geojson).  

In [8]:
conteggi = pois_con_municipio.groupby(['MUNICIPIO', 'geometry', 'tipo']).size().unstack(fill_value=0).reset_index()

Aggiungiamo una colonna "point_etichetta" a cui assegniamo il "centroid" del poligono dei vari municipi, in modo  
da utilizzarlo su kepler per aggiungere un layer "poin" a cui assegniamo una "label" (etichetta) così visualizziamo  
a schermo il numero dell'attributo preso in considerazione (esempio, numero di parchi = 34, così si vede direttamente sulla mappa  
oltre a vedersi solo il colore. Può essere carino).

In [9]:
conteggi["point_etichetta"] = conteggi.geometry.apply(lambda x: x.centroid)

Riconvertiamo in geodataframe e salviamo su file geojson

In [10]:
conteggi = gpd.GeoDataFrame(conteggi, geometry="geometry", crs=CRS_GRAD)
conteggi.to_file(PATH_SCORE_X_MUNICIPI, driver="GeoJSON")

In [16]:
conteggi = gpd.read_file(PATH_SCORE_X_MUNICIPI)
conteggi["parchi"].sum() + conteggi["impianti_sportivi"].sum() + conteggi["aree_gioco"].sum() 

np.int64(1017)